In [32]:
from sklearn.datasets import load_diabetes
import optuna
from catboost import CatBoostClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix, accuracy_score
import seaborn as sn
import pandas as pd

In [10]:
df = pd.read_csv("data/heart.csv")
df.head()

,age,sex,cp,trestbps,chol,fbs,restecg,thalach,exang,oldpeak,slope,ca,thal,target
0,52,1,0,125,212,0,1,168,0,1.0,2,2,3,0
1,53,1,0,140,203,1,0,155,1,3.1,0,0,3,0
2,70,1,0,145,174,0,1,125,1,2.6,0,0,3,0
3,61,1,0,148,203,0,1,161,0,0.0,2,1,3,0
4,62,0,0,138,294,1,1,106,0,1.9,1,3,2,0


In [12]:
df.isna().sum()

age         0
sex         0
cp          0
trestbps    0
chol        0
fbs         0
restecg     0
thalach     0
exang       0
oldpeak     0
slope       0
ca          0
thal        0
target      0
dtype: int64

In [13]:
df.dtypes

age           int64
sex           int64
cp            int64
trestbps      int64
chol          int64
fbs           int64
restecg       int64
thalach       int64
exang         int64
oldpeak     float64
slope         int64
ca            int64
thal          int64
target        int64
dtype: object

In [14]:
X = df.drop("target", axis=1)
y = df["target"]

In [44]:
def objective(trial):
    X_train, X_valid, y_train, y_valid = train_test_split(X, y, test_size=0.3)

    param = {
        "iterations": trial.suggest_int("iterations", 100, 200),
        "depth": trial.suggest_int("depth", 1, 10),
        "l2_leaf_reg": trial.suggest_int("l2_leaf_reg",1, 10),
        "learning_rate": trial.suggest_float("learning_rate", 0.01, 0.1),
    }

    cb = CatBoostClassifier(**param, verbose=False, early_stopping_rounds=5)
    cb.fit(X_train, y_train, eval_set=(X_valid, y_valid))
    accuracy = accuracy_score(y_valid, cb.predict(X_valid))
    return accuracy

In [45]:
study = optuna.create_study(direction="maximize")

[I 2024-04-22 12:41:42,001] A new study created in memory with name: no-name-d805ee18-42d1-4c33-ada5-ab3e676c4ba8


In [46]:
study.optimize(func=objective, n_trials=100)

[W 2024-04-22 12:41:42,363] Trial 0 failed with parameters: {'iterations': 112, 'depth': 6, 'l2_leaf_reg': 3, 'learning_rate': 0.03673521863039162} because of the following error: TypeError("catboost.core.CatBoostClassifier() got multiple values for keyword argument 'verbose'").
Traceback (most recent call last):
  File "C:\Users\Panache Branding\miniconda3\envs\catboost_env\lib\site-packages\optuna\study\_optimize.py", line 196, in _run_trial
    value_or_values = func(trial)
  File "C:\Users\Panache Branding\AppData\Local\Temp\ipykernel_7876\3741104292.py", line 12, in objective
    cb = CatBoostClassifier(**param, verbose=False, early_stopping_rounds=5)
TypeError: catboost.core.CatBoostClassifier() got multiple values for keyword argument 'verbose'
[W 2024-04-22 12:41:42,364] Trial 0 failed with value None.


TypeError: catboost.core.CatBoostClassifier() got multiple values for keyword argument 'verbose'

In [39]:
study.best_params

{'iterations': 179,
 'depth': 9,
 'l2_leaf_reg': 8,
 'learning_rate': 0.07039445922032375}

In [40]:
study.best_value

0.987012987012987